In [1]:
import pandas as pd
import sqlalchemy
# Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 
 
# Create the engine to connect to the PostgreSQL database
engine = sqlalchemy.create_engine('sqlite:///Quick.db')
# https://www.sqlshack.com/introduction-to-sqlalchemy-in-pandas-dataframe/
# Read data from CSV and load into a dataframe object
from sqlalchemy import inspect
inspector = inspect(engine)

In [2]:
# Methord test (make sure the tables are named correctly)
csv_list = ['../Clean_Data/Basketball_Wins_01_18_clean.csv', '../Clean_Data/Football_Wins_01_18_clean.csv',
           '../Clean_Data/MLB_Wins_01_18_Clean.csv', '../Clean_Data/MLS_Wins_01_18_clean.csv', 
            '..Clean_Data/Hockey_Wins_Clean.csv', 
          '../Clean_Data/cleanTitles.csv','../VizTest/EconGDPdf.csv', '../VizTest/FinalGDP.csv',
    '../VizTest/FinalLabor.csv',
            '../US_Can_Econ_Data.csv', '../VizTest/FinalLaborProduct.csv']
name = ['BBall', 'Footie', 'MLB', 'MLS', 'Hockey','titles', 'EconGDP', 'FinalGDP', 
    'FinalLabor', 'EconData', 'LabProduct']

index = 0
for csv in csv_list:
    print(csv)
    print(name[index])
    print('+++++')
    index+=1

../Clean_Data/Basketball_Wins_01_18_clean.csv
BBall
+++++
../Clean_Data/Football_Wins_01_18_clean.csv
Footie
+++++
../Clean_Data/MLB_Wins_01_18_Clean.csv
MLB
+++++
../Clean_Data/MLS_Wins_01_18_clean.csv
MLS
+++++
..Clean_Data/Hockey_Wins_Clean.csv
Hockey
+++++
../Clean_Data/cleanTitles.csv
titles
+++++
../VizTest/EconGDPdf.csv
EconGDP
+++++
../VizTest/FinalGDP.csv
FinalGDP
+++++
../VizTest/FinalLabor.csv
FinalLabor
+++++
../US_Can_Econ_Data.csv
EconData
+++++
../VizTest/FinalLaborProduct.csv
LabProduct
+++++


In [ ]:
# Quick table input into database. DO NOT RERUN WITHOUT MODIFICATION
index = 0
for csv in csv_list:
    data = pd.read_csv(csv)
    name[index]=  data.to_sql(name[index], engine)
    index+=1


In [3]:
# Check input tables
inspector.get_table_names()

['BBall',
 'EconData',
 'EconGDP',
 'FinalLabor',
 'Footie',
 'Hockey',
 'LabProduct',
 'MLB',
 'MLS',
 'titles']

In [4]:
# Check table columns
tables = [inspector.get_table_names()]
for tab in tables[0]:
    columns = inspector.get_columns(tab)
    print('=======================')
    print(tab)
    print('=====================')
    for column in columns:
        print(column["name"], column["type"])


BBall
index BIGINT
Season BIGINT
Team TEXT
Matches BIGINT
Wins BIGINT
Loss BIGINT
Percentage FLOAT
EconData
index BIGINT
Country TEXT
Geography TEXT
VAR TEXT
Variable TEXT
TIME BIGINT
Value BIGINT
EconGDP
index BIGINT
key BIGINT
Sport TEXT
City TEXT
State TEXT
Team TEXT
Season BIGINT
Wins BIGINT
Loss BIGINT
Ties BIGINT
Percentage FLOAT
Country TEXT
Variable TEXT
Value BIGINT
FinalLabor
index BIGINT
key BIGINT
Sport TEXT
City TEXT
State TEXT
Team TEXT
Season BIGINT
Wins BIGINT
Loss BIGINT
Ties BIGINT
Percentage FLOAT
Country TEXT
Variable TEXT
Value BIGINT
Footie
index BIGINT
Season BIGINT
Team TEXT
Wins BIGINT
Loss BIGINT
Ties BIGINT
Percentage FLOAT
Hockey
index BIGINT
year BIGINT
sport TEXT
winner TEXT
winner_metro TEXT
winner_state TEXT
runner_up TEXT
runner_up_metro TEXT
runner_up_state TEXT
final_four3 TEXT
final_four3_metro TEXT
final_four3_state TEXT
final_four4 TEXT
final_four4_metro TEXT
final_four4_state TEXT
LabProduct
index BIGINT
key BIGINT
Sport TEXT
City TEXT
State TEXT


In [5]:
# Test query
from sqlalchemy import text
with engine.connect() as con:
    print(con.execute(text("SELECT City, Season AS 'Year', AVG(Percentage) AS 'Win%', \
    AVG(Value) AS 'Productivity'\
    FROM LabProduct\
    WHERE City =\
        GROUP BY City, Season")).fetchall())

[('Atlanta', 2001, 0.4205, 120789.0), ('Atlanta', 2002, 0.51825, 122340.0), ('Atlanta', 2003, 0.432, 124485.0), ('Atlanta', 2004, 0.48, 127751.0), ('Atlanta', 2005, 0.46825, 129708.0), ('Atlanta', 2006, 0.45399999999999996, 126717.0), ('Atlanta', 2007, 0.40875000000000006, 123595.0), ('Atlanta', 2008, 0.5329999999999999, 120026.0), ('Atlanta', 2009, 0.54175, 123008.0), ('Atlanta', 2010, 0.58175, 123830.0), ('Atlanta', 2011, 0.5933333333333333, 123193.0), ('Atlanta', 2012, 0.6433333333333332, 122820.0), ('Atlanta', 2013, 0.43533333333333335, 124810.0), ('Atlanta', 2014, 0.5316666666666666, 128272.0), ('Atlanta', 2015, 0.49966666666666665, 133952.0), ('Atlanta', 2016, 0.5446666666666666, 134234.0), ('Atlanta', 2017, 0.4875, 132225.0), ('Atlanta', 2018, 0.5135, 134827.0), ('Boston', 2001, 0.556, 138384.0), ('Boston', 2002, 0.539, 138822.0), ('Boston', 2003, 0.5386666666666667, 142605.0), ('Boston', 2004, 0.577, 146543.0), ('Boston', 2005, 0.44733333333333336, 149508.0), ('Boston', 2006, 0

In [17]:
with engine.connect() as con:
    Boston= pd.DataFrame(con.execute(text("SELECT City, Season AS 'Year', AVG(Percentage) AS 'Win%', \
    AVG(Value) AS 'Productivity'\
    FROM LabProduct\
    WHERE City = 'Boston'\
        GROUP BY City, Season")).fetchall())
Boston.head()

,City,Year,Win%,Productivity
0,Boston,2001,0.556000,138384.0
1,Boston,2002,0.539000,138822.0
2,Boston,2003,0.538667,142605.0
3,Boston,2004,0.577000,146543.0
4,Boston,2005,0.447333,149508.0


In [18]:
with engine.connect() as con:
    Dallas= pd.DataFrame(con.execute(text("SELECT City, Season AS 'Year', AVG(Percentage) AS 'Win%', \
    AVG(Value) AS 'Productivity'\
    FROM LabProduct\
    WHERE City = 'Dallas'\
        GROUP BY City, Season")).fetchall())
Dallas.head()

,City,Year,Win%,Productivity
0,Dallas,2001,0.50175,106503.0
1,Dallas,2002,0.56275,110113.0
2,Dallas,2003,0.53025,111319.0
3,Dallas,2004,0.50500,114895.0
4,Dallas,2005,0.62200,117501.0


In [19]:
with engine.connect() as con:
    Pittsburgh= pd.DataFrame(con.execute(text("SELECT City, Season AS 'Year', AVG(Percentage) AS 'Win%', \
    AVG(Value) AS 'Productivity'\
    FROM LabProduct\
    WHERE City ='Pittsburgh'\
        GROUP BY City, Season")).fetchall())
Pittsburgh.head()

,City,Year,Win%,Productivity
0,Pittsburgh,2001,0.528667,110836.0
1,Pittsburgh,2002,0.489667,113779.0
2,Pittsburgh,2003,0.389000,118310.0
3,Pittsburgh,2004,0.692500,121060.0
4,Pittsburgh,2005,0.456667,122175.0


In [15]:
with engine.connect() as con:
    citys =pd.DataFrame(con.execute(text("SELECT City, Season AS 'Year', AVG(Percentage) AS 'Win%', \
    AVG(Value) AS 'Productivity'\
    FROM LabProduct\
    WHERE City IN ('Boston', 'Dallas', 'Pittsburgh', 'Indianapolis', 'San Antonio', 'Charlotte',\
 'Jacksonville', 'Minneapolis', 'Milwaukee', 'Sacramento')  GROUP BY City, Season")).fetchall())
citys

,City,Year,Win%,Productivity
0,Boston,2001,0.556000,138384.0
1,Boston,2002,0.539000,138822.0
2,Boston,2003,0.538667,142605.0
3,Boston,2004,0.577000,146543.0
4,Boston,2005,0.447333,149508.0
...,...,...,...,...
175,San Antonio,2014,0.671000,103660.0
176,San Antonio,2015,0.817000,105620.0
177,San Antonio,2016,0.744000,103841.0
178,San Antonio,2017,0.573000,102558.0


In [ ]:
with engine.connect() as con:
    citys =pd.DataFrame(con.execute(text("SELECT City, Season AS 'Year', AVG(Percentage) AS 'Win%', \
    AVG(Value) AS 'Productivity'\
    FROM LabProduct\
    WHERE City IN ('Boston', 'Dallas', 'Pittsburgh', 'Indianapolis', 'San Antonio', 'Charlotte',\
 'Jacksonville', 'Minneapolis', 'Milwaukee', 'Sacramento')  GROUP BY City, Season")).fetchall())
citys

In [6]:
# DataFrame and csv creation
with engine.connect() as con:
    PerCapYearWins = pd.DataFrame((con.execute(text("SELECT City, \
            Season AS 'Year', AVG(Percentage) AS 'Win%',\
            AVG(Value) AS 'GDP_per_capita'\
            FROM EconGDP\
            GROUP BY City, Season")).fetchall()))
PerCapYearWins.to_csv('PerCapYearWins')
PerCapYearWins.head()

,City,Year,Win%,GDP_per_capita
0,Atlanta,2001,0.42050,64336.0
1,Atlanta,2002,0.51825,63886.0
2,Atlanta,2003,0.43200,64511.0
3,Atlanta,2004,0.48000,65828.0
4,Atlanta,2005,0.46825,66977.0


In [9]:
PerCapYearWins.tail()

,City,Year,Win%,GDP_per_capita
558,Winnipeg,2014,0.524,39730.0
559,Winnipeg,2015,0.427,40622.0
560,Winnipeg,2016,0.488,40385.0
561,Winnipeg,2017,0.634,40922.0
562,Winnipeg,2018,0.573,40801.0


In [7]:
# DataFramen and csv creation
with engine.connect() as con:
    EmployWins = pd.DataFrame(con.execute(text("SELECT City, Season AS 'Year', AVG(Percentage) AS 'Win%', \
        AVG(Value) AS 'Employment'\
        FROM FinalLabor\
        GROUP BY City, Season")).fetchall())
EmployWins.to_csv('EmploymentWins')
EmployWins.head()

,City,Year,Win%,Employment
0,Atlanta,2001,0.42050,2686719.0
1,Atlanta,2002,0.51825,2670925.0
2,Atlanta,2003,0.43200,2687530.0
3,Atlanta,2004,0.48000,2764654.0
4,Atlanta,2005,0.46825,2876385.0


In [26]:
years= [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
 2017, 2018,]

p = figure(width = 400, height = 400)
p.title.text = 'Click on legend entries to mute the corresponding lines'
p = figure(width=800, height=250, x_axis_type="linear")
p.line(years, Boston['Productivity'] , line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="Boston", line_color = 'deepskyblue')
p.line(years, Dallas['Productivity'], line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="Dallas", line_color ='mediumblue')
# p.line(years, city[] , line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="Pittsburgh", line_color ='darkcyan')
# p.line(years, city_main_df.iloc[:,3] , line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="Indianapolis", line_color ='lime')
# p.line(years, city_main_df.iloc[:,4] , line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="San Antonio", line_color ='forestgreen')
# p.line(years, city_main_df.iloc[:,5] , line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="Charlotte", line_color ='coral')
# p.line(years, city_main_df.iloc[:,6] , line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="Jacksonville", line_color ='red')
# p.line(years, city_main_df.iloc[:,7] , line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="Minneapolis", line_color ='orange')
# p.line(years, city_main_df.iloc[:,8] , line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="Milwaukee", line_color ='salmon')
# p.line(years, city_main_df.iloc[:,9] , line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="Sacramento", line_color ='tomato')



p.legend.location = "top_left"
p.legend.click_policy="hide"


show(p)

In [8]:
with engine.connect() as con:
    Productivity = pd.DataFrame(con.execute(text("SELECT City, Season AS 'Year', AVG(Percentage) AS 'Win%', \
    AVG(Value) AS 'Productivity'\
    FROM LabProduct\
        GROUP BY City, Season")).fetchall())
Productivity.to_csv('ProdWins')
Productivity.head()

,City,Year,Win%,Productivity
0,Atlanta,2001,0.42050,120789.0
1,Atlanta,2002,0.51825,122340.0
2,Atlanta,2003,0.43200,124485.0
3,Atlanta,2004,0.48000,127751.0
4,Atlanta,2005,0.46825,129708.0


In [16]:
from bokeh.palettes import Spectral4
from bokeh.plotting import figure, show
from bokeh.layouts import column
from bokeh.models import Arrow, NormalHead, OpenHead, VeeHead
from bokeh.models.annotations import Label
from bokeh.models.annotations import Label
from bokeh.plotting import figure
years= [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
 2017, 2018,]

p = figure(width = 400, height = 400)
p.title.text = 'Click on legend entries to mute the corresponding lines'
p = figure(width=800, height=250, x_axis_type="linear")
#p.line(citys['Year'], city_main_df.iloc[:,0] , line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="Boston", line_color = 'deepskyblue')
p.line(years, citys[citys['City']=='Dallas'], line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="Dallas", line_color ='mediumblue')
# p.line(years, city[] , line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="Pittsburgh", line_color ='darkcyan')
# p.line(years, city_main_df.iloc[:,3] , line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="Indianapolis", line_color ='lime')
# p.line(years, city_main_df.iloc[:,4] , line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="San Antonio", line_color ='forestgreen')
# p.line(years, city_main_df.iloc[:,5] , line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="Charlotte", line_color ='coral')
# p.line(years, city_main_df.iloc[:,6] , line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="Jacksonville", line_color ='red')
# p.line(years, city_main_df.iloc[:,7] , line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="Minneapolis", line_color ='orange')
# p.line(years, city_main_df.iloc[:,8] , line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="Milwaukee", line_color ='salmon')
# p.line(years, city_main_df.iloc[:,9] , line_width=2, alpha=0.8, muted_alpha=0.2, legend_label="Sacramento", line_color ='tomato')


data_label = Label(
    x=2008,  # place the label at the same x coordinates as the recession
    y=90000,  # place the label at the same y coordinates as the circle
    x_offset=12,  # offset the label from the circle center by 12 pixels in the x direction
    text="2008",
    text_baseline="middle",  # align the text vertically to the middle of the circle
    text_color="olive",  # use a standard text property for the text color
)
p.add_layout(data_label)


# data_label = Label(
#     x=2008,  # place the label at the same x coordinates as the recession
#     y=90000,  # place the label at the same y coordinates as the circle
#     x_offset=12,  # offset the label from the circle center by 12 pixels in the x direction
#     text="2008",
#     text_baseline="middle",  # align the text vertically to the middle of the circle
#     text_color="olive",  # use a standard text property for the text color
# )
# p.add_layout(data_label)



arrow = Arrow(
    end=OpenHead(line_color="black", line_width=3),  # 🔁 try using  OpenHead or VeeHead instead of OpenHead
    line_width=3,
    x_start=2008,
    y_start=90000,  # the start of the arrow, defined in data coordinates
    x_end=2008,
    y_end=40000,  # the tip of the arrow is at the minimum of the curve
)
p.add_layout(arrow)

p.legend.location = "top_left"
p.legend.click_policy="mute"


show(p)

SerializationError: can't serialize <class 'pandas.core.frame.DataFrame'>